# Solveur de Jacobi : Modèles de programmation Multi-GPU
Ce notebook présente 12 versions progressives d’un solveur de Jacobi 2D. Chaque section explique le modèle ou l’optimisation, compile la version, exécute et collecte les métriques Nsight.

```markdown
## Modules Spack à charger

Avant de compiler ou d’exécuter les différentes étapes, il est recommandé de charger les modules nécessaires via Spack. Par exemple :

```bash
spack load cuda@12.6
spack load nvidia-nsight-systems@2024.6.1
spack load nvhpc@24.11 
spack load cudnn@9.2.0.82-12
```

Adaptez la version de chaque module selon la configuration de votre cluster.
```

## etape1_cpu
**Description :** Solveur Jacobi CPU de base : implémentation mono-thread. Utile pour valider la correction et les petites tailles de problème ; met en évidence la limite de calcul CPU.

**Intérêt :** Baseline : évalue la limite CPU pour établir une référence.

In [ ]:
%%bash
spack load nvidia-nsight-systems@2024.6.1 cuda@12.6
cd etape1_cpu
make clean all

rm -f main
gcc -O2 -o main main.c
gcc -O2 -o main main.c


In [2]:
%%bash
spack load nvidia-nsight-systems@2024.6.1 cuda@12.6
cd etape1_cpu
nsys profile --stats=true --force-overwrite true -o main ./main

Terminé etape1_cpu
CPU time: 12.633750 seconds
CPU time: 12.633750 seconds
Generating '/tmp/nsys-report-ea89.qdstrm'
Generating '/tmp/nsys-report-ea89.qdstrm'
[1/8] [========================100%] main.nsys-rep
[1/8] [========================100%] main.nsys-rep
[2/8] [========================100%] main.sqlite
[2/8] [========================100%] main.sqlite


SKIPPED: /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite does not contain NV Tools Extension (NVTX) data.
SKIPPED: No data available.
SKIPPED: No data available.
SKIPPED: /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite does not contain CUDA trace data.
SKIPPED: /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite does not contain CUDA trace data.
SKIPPED: /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite does not contain CUDA kernel data.
SKIPPED: /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite does not contain CUDA kernel data.
SKIPPED: /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite does not contain GPU memory data.
SKIPPED: /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite does not contain

[3/8] Executing 'nvtx_sum' stats report
[4/8] Executing 'osrt_sum' stats report
[5/8] Executing 'cuda_api_sum' stats report
[6/8] Executing 'cuda_gpu_kern_sum' stats report
[7/8] Executing 'cuda_gpu_mem_time_sum' stats report
[8/8] Executing 'cuda_gpu_mem_size_sum' stats report
Generated:
    /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.nsys-rep
    /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite
[4/8] Executing 'osrt_sum' stats report
[5/8] Executing 'cuda_api_sum' stats report
[6/8] Executing 'cuda_gpu_kern_sum' stats report
[7/8] Executing 'cuda_gpu_mem_time_sum' stats report
[8/8] Executing 'cuda_gpu_mem_size_sum' stats report
Generated:
    /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.nsys-rep
    /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape1_cpu/main.sqlite


## etape2_cpu_gpu
**Description :** 1 CPU + 1 GPU + 1 stream : le CPU pilote le GPU via un unique stream CUDA. Le calcul Jacobi est entièrement délégué au GPU, le CPU ne fait que l’orchestration.

**Intérêt :** Met en évidence l'écart de performance CPU vs GPU lorsque la grille est suffisamment grande, dans un contexte réaliste d’utilisation d’un seul GPU et d’un seul stream.

### Compilation et exécution (1 CPU + 1 GPU + 1 stream)

In [3]:
%%bash
spack load nvidia-nsight-systems@2024.6.1 cuda@12.6
cd etape2_cpu_gpu
make clean all

rm -f app
nvcc -O2 -o app main.cu kernel.cu
nvcc -O2 -o app main.cu kernel.cu


In [4]:
%%bash
spack load nvidia-nsight-systems@2024.6.1 cuda@12.6
cd etape2_cpu_gpu
nsys profile -t cuda --stats=true --force-overwrite true -o main ./app

Terminé etape2_cpu_gpu (1CPU + 1GPU + 1stream)
GPU time: 1.518556 seconds
GPU time: 1.518556 seconds
Generating '/tmp/nsys-report-dd64.qdstrm'
Generating '/tmp/nsys-report-dd64.qdstrm'
[1/6] [========================100%] main.nsys-rep
[1/6] [========================100%] main.nsys-rep
[2/6] [========================100%] main.sqlite
[2/6] [========================100%] main.sqlite
[3/6] Executing 'cuda_api_sum' stats report

 Time (%)  Total Time (ns)  Num Calls   Avg (ns)     Med (ns)    Min (ns)   Max (ns)   StdDev (ns)           Name         
 --------  ---------------  ---------  -----------  -----------  ---------  ---------  -----------  ----------------------
     73.4        706122688          3  235374229.3  351699424.0    1335456  353087808  202684712.0  cudaMemcpyAsync       
     15.7        150938016          1  150938016.0  150938016.0  150938016  150938016          0.0  cudaStreamCreate      
     10.4         99869248       1001      99769.5      98304.0       1728    

## etape3_mpi_gpus
**Description :** MPI + GPUs : domaine réparti sur plusieurs rangs MPI, chacun pilotant un GPU. Illustrations des défis de mise à l'échelle multi-nœuds et des communications inter-rangs.

**Intérêt :** Test de montée en charge inter-nœuds et coût MPI sur cluster multi-GPU.

**Step :** 
- Initialiser MPI, déterminer le rang et le nombre de processus.
- Associer chaque rang à un GPU différent.
- Diviser la grille entre les rangs (découpage 1D vertical).
- Gérer les échanges d’halos entre rangs voisins avec MPI_Sendrecv.
- Synchroniser les échanges à chaque itération.
- Nettoyer MPI à la fin.

In [14]:
%%bash
spack load nvidia-nsight-systems@2024.6.1 cuda@12.6 nvhpc@24.11
cd etape3_mpi_gpus
make clean all

rm -f app
nvcc -O2 -lcudart -lmpi -lnccl -lstdc++ -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/include -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/include -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/include -Xlinker --no-as-needed -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/lib64 -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/lib -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/lib -o app main.cu kernel.cu
nvcc -O2 -lcudart -lmpi -lnccl -lstdc++ -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/include -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/include -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/include -Xlinker --no-as-needed -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/2

In [ ]:
%%bash
spack load nvidia-nsight-systems@2024.6.1 cuda@12.6 nvhpc@24.11
export LD_LIBRARY_PATH=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/lib:/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/lib:$LD_LIBRARY_PATH
unset OPAL_PREFIX
unset PMIX_INSTALL_PREFIX
cd etape3_mpi_gpus
nsys profile -t mpi,cuda --stats=true --force-overwrite true -o main mpirun -np 4 ./app

Lancement de l'étape 3 : MPI + multi-GPU
Grille de taille 4096 x 4096, T = 1000 itérations
Grille de taille 4096 x 4096, T = 1000 itérations
Terminé etape3_mpi_gpus (MPI + multi-GPU) rank 0/4
Max GPU time: 0.573540 seconds rank 0/4
Terminé etape3_mpi_gpus (MPI + multi-GPU) rank 0/4
Max GPU time: 0.573540 seconds rank 0/4
Generating '/tmp/nsys-report-1c77.qdstrm'
Generating '/tmp/nsys-report-1c77.qdstrm'
[1/7] [========================100%] main.nsys-rep
[1/7] [========================100%] main.nsys-rep
[2/7] [========================100%] main.sqlite
[2/7] [========================100%] main.sqlite


SKIPPED: No data available.


[3/7] Executing 'nvtx_sum' stats report
[4/7] Executing 'cuda_api_sum' stats report

 Time (%)  Total Time (ns)  Num Calls  Avg (ns)   Med (ns)  Min (ns)  Max (ns)  StdDev (ns)           Name         
 --------  ---------------  ---------  ---------  --------  --------  --------  -----------  ----------------------
     84.1        787842048      12012    65587.9    8224.0      3168  88112512    2256311.0  cudaMemcpy            
     12.9        120601856       4000    30150.5   29504.0      3680   2358400      36872.6  cudaDeviceSynchronize 
      1.6         15403584       4000     3850.9    3104.0      2080    518208      16143.0  cudaLaunchKernel      
      0.9          8369792          8  1046224.0  567360.0    486528   2878880     912372.8  cudaMalloc            
      0.4          3630144          8   453768.0  519568.0     84416    679008     208549.3  cudaFree              
      0.0           435360       1652      263.5     128.0        32      5216        312.2  cuGetProcA

## etape4_mpi_overlap
**Description :** MPI + recouvrement : recouvrements des échanges d’halo non-bloquants avec le calcul Jacobi local. Réduit l'impact de la latence réseau.

**Intérêt :** Cache la latence réseau en recouvrant communication et calcul local.

In [19]:
%%bash
spack load nvidia-nsight-systems@2024.6.1 cuda@12.6 nvhpc@24.11
cd etape4_mpi_overlap
make clean all

rm -f app
nvcc -O2 -lcudart -lmpi -lnccl -lstdc++ -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/include -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/include -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/include -Xlinker --no-as-needed -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.6/lib64 -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/lib -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/lib -o app main.cu kernel.cu


In [26]:
%%bash 
cd etape4_mpi_overlap
spack load nvidia-nsight-systems@2024.6.1 cuda@12.6 nvhpc@24.11
export LD_LIBRARY_PATH=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/hpcx/hpcx-2.20/ompi/lib:/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/nccl/lib:$LD_LIBRARY_PATH
unset OPAL_PREFIX
unset PMIX_INSTALL_PREFIX
nsys profile -t mpi,cuda --stats=true --force-overwrite true -o main mpirun -np 4 ./app

Lancement de l'étape 3 : MPI + multi-GPU
Grille de taille 4096 x 4096, T = 1000 itérations
Terminé etape3_mpi_gpus (MPI + multi-GPU) rank 0/4
Max GPU time: 0.794325 seconds rank 0/4
Generating '/tmp/nsys-report-e8bb.qdstrm'
[1/7] [========================100%] main.nsys-rep
[2/7] [========================100%] main.sqlite


SKIPPED: No data available.
SKIPPED: /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape4_mpi_overlap/main.sqlite does not contain CUDA kernel data.
SKIPPED: /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape4_mpi_overlap/main.sqlite does not contain GPU memory data.
SKIPPED: /gpfs/home/colevalet/Cours/CHPS0904_RENDU/MultiGPU Programming Model/etape4_mpi_overlap/main.sqlite does not contain GPU memory data.


[3/7] Executing 'nvtx_sum' stats report
[4/7] Executing 'cuda_api_sum' stats report

 Time (%)  Total Time (ns)  Num Calls  Avg (ns)  Med (ns)  Min (ns)  Max (ns)   StdDev (ns)           Name         
 --------  ---------------  ---------  --------  --------  --------  ---------  -----------  ----------------------
     63.0       1214923168       8000  151865.4     256.0       160  322465440    6794687.2  cudaDeviceSynchronize 
     36.5        704306144      12012   58633.5     288.0       160   88106560    2257784.4  cudaMemcpy            
      0.3          5333856      10000     533.4     320.0       192     594144      10022.3  cudaLaunchKernel      
      0.1          2684928          8  335616.0  306272.0    146464     598368     148155.5  cudaMalloc            
      0.0           367776       1652     222.6      96.0        32       4704        278.2  cuGetProcAddress_v2   
      0.0            89824          8   11228.0   10656.0      2432      18592       7148.0  cudaFree  

## etape5_nccl
**Description :** NCCL : utilisation de la NVIDIA Collective Communications Library pour les échanges GPU à GPU. Montre les gains via NVLink ou PCIe haute bande passante.

**Intérêt :** Exploite automatiquement le topologie NVLink/PCIe pour des échanges GPU efficaces.

In [ ]:
%%bash
cd etape5_nccl
make all

In [ ]:
%%bash 
cd etape5_nccl
nv-nsight-cu-cli --csv --report-file rapport_etape5_nccl.csv ./main
cat rapport_etape5_nccl.csv

## etape6_nccl_overlap
**Description :** NCCL + recouvrement : superposition des collectifs NCCL avec le calcul sur GPU, cachant le coût de communication.

**Intérêt :** Essentiel à forte densité GPU pour maintenir les cœurs occupés.

In [ ]:
%%bash 
cd etape6_nccl_overlap
make all

In [ ]:
%%bash 
cd etape6_nccl_overlap 
nv-nsight-cu-cli --csv --report-file rapport_etape6_nccl_overlap.csv ./main
cat rapport_etape6_nccl_overlap.csv

## etape7_nccl_graphs
**Description :** NCCL + CUDA Graphs : capture et relecture des séquences Jacobi/échange pour réduire le surcoût des lancements.

**Intérêt :** Réduit l’overhead de lancement grâce aux CUDA Graphs.

In [ ]:
%%bash 
cd etape7_nccl_graphs
make all

In [ ]:
%%bash 
cd etape7_nccl_graphs
nv-nsight-cu-cli --csv --report-file rapport_etape7_nccl_graphs.csv ./main
cat rapport_etape7_nccl_graphs.csv

## etape8_nvshmem
**Description :** NVSHMEM : modèle PGAS à accès mémoire unilatéral GPU, simplifiant les mises à jour d’halo.

**Intérêt :** Simplifie les échanges via modèle PGAS unilatéral.

In [ ]:
%%bash 
cd etape8_nvshmem
make all

In [ ]:
%%bash 
cd etape8_nvshmem
nv-nsight-cu-cli --csv --report-file rapport_etape8_nvshmem.csv ./main
cat rapport_etape8_nvshmem.csv

## etape9_nvshmem_lt
**Description :** NVSHMEM + LTO : ajout de l’optimisation link-time pour inliner les fonctions critiques et réduire le coût des appels.

**Intérêt :** Optimisation link-time pour inliner les sections critiques.

In [ ]:
%%bash cd etape9_nvshmem_lt
make all

In [ ]:
%%bash 
cd etape9_nvshmem_lt
nv-nsight-cu-cli --csv --report-file rapport_etape9_nvshmem_lt.csv ./main
cat rapport_etape9_nvshmem_lt.csv

## etape10_vshmem_neighborhood_lto
**Description :** vshmem neighborhood_sync + LTO : synchronisation fine-grain de voisinage et optimisations link-time O2.

**Intérêt :** Synchronisation fine et LTO pour boucles serrées.

In [ ]:
%%bash 
cd etape10_vshmem_neighborhood_lto
make all

In [ ]:
%%bash 
cd etape10_vshmem_neighborhood_lto
nv-nsight-cu-cli --csv --report-file rapport_etape10_vshmem_neighborhood_lto.csv ./main 
cat rapport_etape10_vshmem_neighborhood_lto.csv

## etape11_nvshmem_norm_overlap_neighborhood_sync_lto
**Description :** Combinaison : NVSHMEM avec recouvrement, synchrone de voisinage, et LTO pour maximiser la concurrence.

**Intérêt :** Combinaison des meilleures pratiques pour un binaire ultra-optimisé.

In [ ]:
%%bash cd etape11_nvshmem_norm_overlap_neighborhood_sync_lto
make all

In [ ]:
%%bash cd etape11_nvshmem_norm_overlap_neighborhood_sync_lto
nv-nsight-cu-cli --csv --report-file rapport_etape11_nvshmem_norm_overlap_neighborhood_sync_lto.csv ./main
cat rapport_etape11_nvshmem_norm_overlap_neighborhood_sync_lto.csv

## etape12_nvshmem_norm_overlap_neighborhood_sync_lto_ext1
**Description :** Tuning étendu : paramètres ajustables (taille de tuile, ordre de boucles) et hooks de benchmark.

**Intérêt :** Ajout de paramètres de tuning et hooks de benchmarking.

In [ ]:
%%bash 
cd etape12_nvshmem_norm_overlap_neighborhood_sync_lto_ext1 
make all

In [ ]:
%%bash 
cd etape12_nvshmem_norm_overlap_neighborhood_sync_lto_ext1 
nv-nsight-cu-cli --csv --report-file rapport_etape12_nvshmem_norm_overlap_neighborhood_sync_lto_ext1.csv ./main
cat rapport_etape12_nvshmem_norm_overlap_neighborhood_sync_lto_ext1.csv